In [62]:
import subprocess
from time import sleep
from mss import mss
from PIL import Image

In [63]:
rs = "RuneScape"

In [64]:
def search_windows_by_name(name):
    try:
        return subprocess.check_output(["xdotool", "search", "--name", name]).splitlines()
    except:
        return []
    
def activate_window(window):
    if window:
        return subprocess.call(["xdotool", "windowactivate", window])

def get_window_geometry(window):
    if not window:
        return ((10,10),(10,10))
    
    stats = subprocess.check_output(["xdotool", "getwindowgeometry", window]).splitlines()
    geometry = 0
    position = 0
    for _stat in stats:
        stat = _stat.decode("utf-8").strip()
        if stat.startswith("Position: "):
            stat = stat[stat.find(" "):].strip()
            stat = stat[:stat.find(" ")].strip()
            position = stat.split(",")
            position = (int(position[0]), int(position[1]))
        if stat.startswith("Geometry: "):
            stat = stat[stat.find(" "):].strip()
            geometry = stat.split("x")
            geometry = (int(geometry[0]), int(geometry[1]))

    return (position, geometry)     

In [65]:
def get_game_window():
    for window in search_windows_by_name(rs):
        if get_window_geometry(window) != ((10,10),(10,10)):
            return window
    return None

def get_window_bbox(window):
    if not window:
        return None
    geometry = get_window_geometry(window)
    left = geometry[0][0]
    top = geometry[0][1] - 19
    right = left + geometry[1][0]
    bottom = top + geometry[1][1]
    return (left, top, right, bottom)    
    
def get_window_viewport(window):
    bbox = get_window_bbox(window)
    if not bbox:
        return None
    
    with mss() as sct:
        monitor = sct.monitors[1]
        sct_img = sct.grab(bbox)
        return Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')

In [66]:
def get_current_frame():
    game_window = get_game_window()
    if not game_window:
        return None
    
    activate_window(game_window)
    return get_window_viewport(game_window)

In [69]:
get_current_frame()